<a href="https://colab.research.google.com/github/ninawang4/mbti-classifier/blob/main/Separated_Distilbert_Multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
rm -rf 6864-hw3
git clone https://github.com/mit-6864/hw3.git

Cloning into 'hw3'...


In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import pandas as pd

with open('/content/drive/MyDrive/NLP_project/Data/original_seperated.csv', 'r', encoding='utf-8') as f:
    df = pd.DataFrame(csv.reader(f))
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    print(df.head())

0                        caption    username  mbti      group    name  \
0                            📸\n  suye.on2di  ISTJ  Weki Meki  Suyeon   
1  제 손이 또...커튼을 망가트리고 말았습니다...\n  suye.on2di  ISTJ  Weki Meki  Suyeon   
2                      Synergy\n  suye.on2di  ISTJ  Weki Meki  Suyeon   
3                         해보자고\n  suye.on2di  ISTJ  Weki Meki  Suyeon   
4                         🌻😁❣️\n  suye.on2di  ISTJ  Weki Meki  Suyeon   

0 extraversion sensing thinking judging  
0            0       1        1       1  
1            0       1        1       1  
2            0       1        1       1  
3            0       1        1       1  
4            0       1        1       1  


In [ ]:
import sys
# sys.path.append("/content/hw3")

import csv
import itertools as it
import numpy as np
import sklearn.decomposition
np.random.seed(0)
from tqdm import tqdm

# import lab_util

In [ ]:
# from transformers import AutoTokenizer
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import TensorDataset
def tokenizer_function(input_data, labels):
  input_ids = []
  attention_masks = []
  for sent in input_data:
    this_encoding = tokenizer.encode_plus(sent, truncation=True, pad_to_max_length = True,max_length = 512,return_attention_mask = True,return_tensors = 'pt')
    input_ids.append(this_encoding['input_ids'])
    attention_masks.append( this_encoding['attention_mask'])
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  tokenized_data = TensorDataset(input_ids, attention_masks, labels)
  return tokenized_data

In [ ]:
import numpy as np
np.random.seed(0)

# shuffle dataset before splitting
data = df['caption']
extraversion = df['extraversion'].astype(int)
sensing = df['sensing'].astype(int)
thinking = df['thinking'].astype(int)
judging = df['judging'].astype(int)


obs = list(zip(data, extraversion, sensing, thinking, judging))
np.random.shuffle(obs)

captions, extraversion, sensing, thinking, judging = zip(*obs)



In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

## Sanity Check

In [ ]:
print(captions[:3])
assert set(extraversion) == {0,1}
assert set(sensing) == {0,1}
assert set(thinking) == {0,1}
assert set(judging) == {0,1}

('#GG Band Rehearsals 🎤🎹🎸🎷🎤🎶 #다시만난세계 #유스케 #감동\n', 'New video tomorrow~ 😎❤️ #everyfriday\n', '누가 나 자는데 때렷냐😵\n')


## Extraversion/Introversion

In [ ]:
# 70% train, 15% test, 15% validation
subsample = 20

train_dataset_E = tokenizer_function(captions[0:84500:subsample], extraversion[0:84500:subsample])
val_dataset_E = tokenizer_function(captions[84500:102500:subsample], extraversion[84500:102500:subsample])
test_dataset_E = tokenizer_function(captions[102500::subsample], extraversion[102500::subsample])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
import torch.nn.functional as F
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

model_E = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2, output_attentions = False, output_hidden_states=False)
model_E.to(device)

train_loader = DataLoader(train_dataset_E, batch_size=32, shuffle=True) #Feel free to experiment with batch sizes
val_loader = DataLoader(val_dataset_E, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_E, batch_size=32, shuffle=True)

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model_E.parameters(),
                lr = 5e-5, # args.learning_rate - default is 5e-5.
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
              )


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Now it is time to train. Notice how it is working, it might potentially help with your project as well!

In [ ]:
total_t0 = time.time()
for epoch in tqdm(range(3)):
  total_train_loss = 0
  model_E.train()
  for step, batch in enumerate(train_loader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    model_E.zero_grad()
    outputs = model_E(b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_labels)
    loss = outputs.loss
    total_train_loss += loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model_E.parameters(), 1.0)
    optimizer.step()
    if(step %10 == 0):
      print(loss)

  avg_train_loss = total_train_loss / len(train_loader)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))

  print("")
  print("Running Validation...")

  t0 = time.time()

  # Put the model in evaluation mode-
  model_E.eval()

  # Tracking variables
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0

    # Evaluate data for one epoch
  for batch in val_loader:
      #
      # `batch` contains three pytorch tensors:
      #   [0]: input ids
      #   [1]: attention masksx
      #   [2]: labels
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():

          outputs = model_E(b_input_ids,
                                  attention_mask=b_input_mask,
                                  labels=b_labels)

      # Accumulate the validation loss.
      loss = outputs.loss
      logits = outputs.logits
      total_eval_loss += loss.item()

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      # Calculate the accuracy for this batch of test sentences, and
      # accumulate it over all batches.
      total_eval_accuracy += flat_accuracy(logits, label_ids)


  # Report the final accuracy for this validation run.
  avg_val_accuracy = total_eval_accuracy / len(val_loader)
  print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

  # Calculate the average loss over all of the batches.
  avg_val_loss = total_eval_loss / len(val_loader)

  # Measure how long the validation run took.
  validation_time = format_time(time.time() - t0)

  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:}".format(format_time(time.time()-total_t0)))

  0%|          | 0/3 [00:00<?, ?it/s]

tensor(0.6886, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7128, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6839, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6976, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6837, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6874, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7035, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6836, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7153, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.69

Running Validation...


 33%|███▎      | 1/3 [03:16<06:32, 196.30s/it]

  Accuracy: 0.53
  Validation Loss: 0.69
  Validation took: 0:00:14
tensor(0.6511, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7212, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6420, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6018, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6808, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6827, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6718, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6785, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6419, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6884, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6384, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6763, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6075, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6790, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.69

Running Validation...


 67%|██████▋   | 2/3 [06:37<03:19, 199.42s/it]

  Accuracy: 0.54
  Validation Loss: 0.69
  Validation took: 0:00:14
tensor(0.6277, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6347, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6858, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6676, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5628, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6394, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6223, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6422, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5843, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7376, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5856, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6089, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6673, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.63

Running Validation...


100%|██████████| 3/3 [09:59<00:00, 199.89s/it]

  Accuracy: 0.57
  Validation Loss: 0.72
  Validation took: 0:00:15

Training complete!
Total training took 0:10:00


Now we can check how our accuracy is!

In [ ]:
print('Predicting labels for test sentences...')

# Put model in evaluation mode
model_E.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in test_loader:

  # Unpack the inputs from our dataloader
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model_E(b_input_ids,
                      attention_mask=b_input_mask)
  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.extend(logits)
  true_labels.extend(label_ids)
print('Final Accuracy: {0}'.format(flat_accuracy(np.asarray(predictions), np.asarray(true_labels))))

print('    DONE.')

Predicting labels for test sentences...
tensor([[   101,    146,    100,  ...,      0,      0,      0],
        [   101,    164,   9625,  ...,      0,      0,      0],
        [   101,  12489,  10635,  ...,      0,      0,      0],
        ...,
        [   101,  39773, 102574,  ...,      0,      0,      0],
        [   101,   9651, 119008,  ...,      0,      0,      0],
        [   101,  30085,  37712,  ...,      0,      0,      0]],
       device='cuda:0')
tensor([[  101,   119,   164,  ...,     0,     0,     0],
        [  101,  1910, 26554,  ...,     0,     0,     0],
        [  101, 11590, 14687,  ...,     0,     0,     0],
        ...,
        [  101,  9599,  9379,  ...,     0,     0,     0],
        [  101,   164,   108,  ...,     0,     0,     0],
        [  101, 12242, 11951,  ...,     0,     0,     0]], device='cuda:0')
tensor([[  101,   108,  9074,  ...,     0,     0,     0],
        [  101,   108,   168,  ...,     0,     0,     0],
        [  101,   125, 10954,  ...,     0, 

KeyboardInterrupt: ignored

## Sensing/Intuition

In [ ]:
# 70% train, 15% test, 15% validation
subsample = 20

train_dataset_S = tokenizer_function(captions[0:84500:subsample], sensing[0:84500:subsample])
val_dataset_S = tokenizer_function(captions[84500:102500:subsample], sensing[84500:102500:subsample])
test_dataset_S = tokenizer_function(captions[102500::subsample], sensing[102500::subsample])

In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
import torch.nn.functional as F
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

model_S = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2, output_attentions = False, output_hidden_states=False)
model_S.to(device)

train_loader = DataLoader(train_dataset_S, batch_size=32, shuffle=True) #Feel free to experiment with batch sizes
val_loader = DataLoader(val_dataset_S, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_S, batch_size=32, shuffle=True)

In [ ]:
optimizer = AdamW(model_S.parameters(),
                lr = 5e-5, # args.learning_rate - default is 5e-5.
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
              )


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Now it is time to train. Notice how it is working, it might potentially help with your project as well!

In [ ]:
total_t0 = time.time()
for epoch in tqdm(range(3)):
  total_train_loss = 0
  model_S.train()
  for step, batch in enumerate(train_loader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    model_S.zero_grad()
    outputs = model_S(b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_labels)
    loss = outputs.loss
    total_train_loss += loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model_S.parameters(), 1.0)
    optimizer.step()
    if(step %10 == 0):
      print(loss)

  avg_train_loss = total_train_loss / len(train_loader)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))

  print("")
  print("Running Validation...")

  t0 = time.time()

  # Put the model in evaluation mode-
  model_S.eval()

  # Tracking variables
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0

    # Evaluate data for one epoch
  for batch in val_loader:
      #
      # `batch` contains three pytorch tensors:
      #   [0]: input ids
      #   [1]: attention masksx
      #   [2]: labels
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():

          outputs = model_S(b_input_ids,
                                  attention_mask=b_input_mask,
                                  labels=b_labels)

      # Accumulate the validation loss.
      loss = outputs.loss
      logits = outputs.logits
      total_eval_loss += loss.item()

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      # Calculate the accuracy for this batch of test sentences, and
      # accumulate it over all batches.
      total_eval_accuracy += flat_accuracy(logits, label_ids)


  # Report the final accuracy for this validation run.
  avg_val_accuracy = total_eval_accuracy / len(val_loader)
  print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

  # Calculate the average loss over all of the batches.
  avg_val_loss = total_eval_loss / len(val_loader)

  # Measure how long the validation run took.
  validation_time = format_time(time.time() - t0)

  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:}".format(format_time(time.time()-total_t0)))

  0%|          | 0/3 [00:00<?, ?it/s]

tensor(0.6355, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6880, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6552, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6433, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6348, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6451, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6398, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6492, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6480, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6538, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6444, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6476, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6250, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.66

Running Validation...


 33%|███▎      | 1/3 [03:23<06:47, 203.58s/it]

  Accuracy: 0.66
  Validation Loss: 0.66
  Validation took: 0:00:15
tensor(0.6681, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6742, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7034, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6724, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6767, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6297, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6957, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6658, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6983, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6580, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6712, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6218, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6691, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6339, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.66

Running Validation...


 67%|██████▋   | 2/3 [06:46<03:23, 203.24s/it]

  Accuracy: 0.65
  Validation Loss: 0.66
  Validation took: 0:00:15
tensor(0.6709, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6732, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6347, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6685, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6714, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6718, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6605, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6657, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6513, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6397, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7108, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6677, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6237, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.66

Running Validation...


100%|██████████| 3/3 [10:09<00:00, 203.26s/it]

  Accuracy: 0.66
  Validation Loss: 0.66
  Validation took: 0:00:15

Training complete!
Total training took 0:10:10


Now we can check how our accuracy is!

In [ ]:
print('Predicting labels for test sentences...')

# Put model in evaluation mode
model_S.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in test_loader:

  # Unpack the inputs from our dataloader
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model_S(b_input_ids,
                      attention_mask=b_input_mask)
  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.extend(logits)
  true_labels.extend(label_ids)
print('Final Accuracy: {0}'.format(flat_accuracy(np.asarray(predictions), np.asarray(true_labels))))

print('    DONE.')

Predicting labels for test sentences...
Final Accuracy: 0.7008456659619451
    DONE.


## Thinking/Feeling

In [ ]:
# 70% train, 15% test, 15% validation
subsample = 20

train_dataset_T = tokenizer_function(captions[0:84500:subsample], thinking[0:84500:subsample])
val_dataset_T = tokenizer_function(captions[84500:102500:subsample], thinking[84500:102500:subsample])
test_dataset_T = tokenizer_function(captions[102500::subsample], thinking[102500::subsample])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
import torch.nn.functional as F
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

model_T = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2, output_attentions = False, output_hidden_states=False)
model_T.to(device)

train_loader = DataLoader(train_dataset_T, batch_size=32, shuffle=True) #Feel free to experiment with batch sizes
val_loader = DataLoader(val_dataset_T, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_T, batch_size=32, shuffle=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model_T.parameters(),
                lr = 5e-5, # args.learning_rate - default is 5e-5.
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
              )


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Now it is time to train. Notice how it is working, it might potentially help with your project as well!

In [ ]:
total_t0 = time.time()
for epoch in tqdm(range(3)):
  total_train_loss = 0
  model_T.train()
  for step, batch in enumerate(train_loader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    model_T.zero_grad()
    outputs = model_T(b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_labels)
    loss = outputs.loss
    total_train_loss += loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model_T.parameters(), 1.0)
    optimizer.step()
    if(step %10 == 0):
      print(loss)

  avg_train_loss = total_train_loss / len(train_loader)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))

  print("")
  print("Running Validation...")

  t0 = time.time()

  # Put the model in evaluation mode-
  model_T.eval()

  # Tracking variables
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0

    # Evaluate data for one epoch
  for batch in val_loader:
      #
      # `batch` contains three pytorch tensors:
      #   [0]: input ids
      #   [1]: attention masksx
      #   [2]: labels
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():

          outputs = model_T(b_input_ids,
                                  attention_mask=b_input_mask,
                                  labels=b_labels)

      # Accumulate the validation loss.
      loss = outputs.loss
      logits = outputs.logits
      total_eval_loss += loss.item()

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      # Calculate the accuracy for this batch of test sentences, and
      # accumulate it over all batches.
      total_eval_accuracy += flat_accuracy(logits, label_ids)


  # Report the final accuracy for this validation run.
  avg_val_accuracy = total_eval_accuracy / len(val_loader)
  print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

  # Calculate the average loss over all of the batches.
  avg_val_loss = total_eval_loss / len(val_loader)

  # Measure how long the validation run took.
  validation_time = format_time(time.time() - t0)

  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:}".format(format_time(time.time()-total_t0)))

  0%|          | 0/3 [00:00<?, ?it/s]

tensor(0.6811, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5188, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6103, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5061, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5710, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6382, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7009, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6389, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5223, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6731, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6433, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6269, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6432, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.61

Running Validation...


 33%|███▎      | 1/3 [03:18<06:36, 198.38s/it]

  Accuracy: 0.70
  Validation Loss: 0.60
  Validation took: 0:00:15
tensor(0.6075, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6410, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5716, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5332, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5526, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3503, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5363, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5811, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7322, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6045, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5558, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5581, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5141, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.58

Running Validation...


 67%|██████▋   | 2/3 [06:40<03:20, 200.40s/it]

  Accuracy: 0.72
  Validation Loss: 0.60
  Validation took: 0:00:15
tensor(0.7362, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4688, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5888, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5838, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5093, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4785, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4173, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5066, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4065, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3612, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5323, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4346, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5191, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.47

Running Validation...


100%|██████████| 3/3 [10:02<00:00, 200.69s/it]

  Accuracy: 0.71
  Validation Loss: 0.63
  Validation took: 0:00:15

Training complete!
Total training took 0:10:02


Now we can check how our accuracy is!

In [ ]:
print('Predicting labels for test sentences...')

# Put model in evaluation mode
model_T.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in test_loader:

  # Unpack the inputs from our dataloader
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model_T(b_input_ids,
                      attention_mask=b_input_mask)
  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.extend(logits)
  true_labels.extend(label_ids)
print('Final Accuracy: {0}'.format(flat_accuracy(np.asarray(predictions), np.asarray(true_labels))))

print('    DONE.')

Predicting labels for test sentences...
Final Accuracy: 0.7066596194503171
    DONE.


## Judging/Perceiving

In [ ]:
# 70% train, 15% test, 15% validation
subsample = 20

train_dataset_J = tokenizer_function(captions[0:84500:subsample], judging[0:84500:subsample])
val_dataset_J = tokenizer_function(captions[84500:102500:subsample], judging[84500:102500:subsample])
test_dataset_J = tokenizer_function(captions[102500::subsample], judging[102500::subsample])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
import torch.nn.functional as F
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

model_J = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2, output_attentions = False, output_hidden_states=False)
model_J.to(device)

train_loader = DataLoader(train_dataset_J, batch_size=32, shuffle=True) #Feel free to experiment with batch sizes
val_loader = DataLoader(val_dataset_J, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_J, batch_size=32, shuffle=True)

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model_J.parameters(),
                lr = 5e-5, # args.learning_rate - default is 5e-5.
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
              )


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Now it is time to train. Notice how it is working, it might potentially help with your project as well!

In [ ]:
total_t0 = time.time()
for epoch in tqdm(range(3)):
  total_train_loss = 0
  model_J.train()
  for step, batch in enumerate(train_loader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    model_J.zero_grad()
    outputs = model_J(b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_labels)
    loss = outputs.loss
    total_train_loss += loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model_J.parameters(), 1.0)
    optimizer.step()
    if(step %10 == 0):
      print(loss)

  avg_train_loss = total_train_loss / len(train_loader)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))

  print("")
  print("Running Validation...")

  t0 = time.time()

  # Put the model in evaluation mode-
  model_J.eval()

  # Tracking variables
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0

    # Evaluate data for one epoch
  for batch in val_loader:
      #
      # `batch` contains three pytorch tensors:
      #   [0]: input ids
      #   [1]: attention masksx
      #   [2]: labels
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():

          outputs = model_J(b_input_ids,
                                  attention_mask=b_input_mask,
                                  labels=b_labels)

      # Accumulate the validation loss.
      loss = outputs.loss
      logits = outputs.logits
      total_eval_loss += loss.item()

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      # Calculate the accuracy for this batch of test sentences, and
      # accumulate it over all batches.
      total_eval_accuracy += flat_accuracy(logits, label_ids)


  # Report the final accuracy for this validation run.
  avg_val_accuracy = total_eval_accuracy / len(val_loader)
  print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

  # Calculate the average loss over all of the batches.
  avg_val_loss = total_eval_loss / len(val_loader)

  # Measure how long the validation run took.
  validation_time = format_time(time.time() - t0)

  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:}".format(format_time(time.time()-total_t0)))

  0%|          | 0/3 [00:00<?, ?it/s]

tensor(0.6418, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6190, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6261, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5266, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7213, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6029, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5541, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5405, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6686, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5838, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7089, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7006, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6497, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.63

Running Validation...


 33%|███▎      | 1/3 [03:21<06:43, 201.74s/it]

  Accuracy: 0.59
  Validation Loss: 0.64
  Validation took: 0:00:14
tensor(0.6432, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6036, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6472, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6390, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6334, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3726, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3361, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4014, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4266, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5021, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5792, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4854, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.52

Running Validation...


 67%|██████▋   | 2/3 [06:43<03:21, 201.68s/it]

  Accuracy: 0.60
  Validation Loss: 0.68
  Validation took: 0:00:14
tensor(0.4428, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0995, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3171, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1719, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3232, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4346, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2384, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3662, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3433, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4453, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3286, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4480, device='cuda:0', grad_fn=<NllLossBackward0>)

  Average training loss: 0.36

Running Validation...


100%|██████████| 3/3 [10:05<00:00, 201.68s/it]

  Accuracy: 0.58
  Validation Loss: 0.91
  Validation took: 0:00:14

Training complete!
Total training took 0:10:05


Now we can check how our accuracy is!

In [ ]:
print('Predicting labels for test sentences...')

# Put model in evaluation mode
model_J.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in test_loader:

  # Unpack the inputs from our dataloader
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model_J(b_input_ids,
                      attention_mask=b_input_mask)
  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.extend(logits)
  true_labels.extend(label_ids)
print('Final Accuracy: {0}'.format(flat_accuracy(np.asarray(predictions), np.asarray(true_labels))))

print('    DONE.')

Predicting labels for test sentences...
Final Accuracy: 0.612691466083151
    DONE.


## Base Rates

In [ ]:
print(sum(extraversion)/len(extraversion),
      sum(sensing)/len(sensing),
      sum(thinking)/len(thinking),
      sum(judging)/len(judging)
      )

0.5165464755009549 0.3143687826012599 0.295755779152296 0.5284967078072


## Results
distilbert-base-multilingual-cased, 5% subsample: E/I 0.599 acc, S/I 0.705 acc, T/F 0.700 acc, J/P 0.613 acc